# Dimensionality reduction 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Principal Component Analysis
- Linear dimensionality reduction method
- Find optimal orthogonal transformation such that covariance between the new dimensions is 0
    - Exploits eigen decomposition
    - By the orthogonal transformation, we could ignore the low variance direction to reduce the dimensionality of the features.
- Transformations
    - Find the basis (eigen vector of covariance matrix) such that independent each other
    - Diagonalization by changing the basis (inner-product of the original matrix and basis (new coordinate system)) 
    $$
    \mathbf{X} \text{: Data original coordinates}
    \\
    \hat{\mathbf{X}} \text{: Centralized dataset}
    \\
    \mathbf{\Gamma} \text{: New coordinate system (Eigen vectors matrix)}
    \\
    \mathbf{\Lambda} \text{: Variance of respective direction (Eigen values matrix)}
    \\
    $$
    Eigen Decomposition of covariance matrix of original data
    $$
    \Sigma_{\hat{\mathbf{X}}} =  \mathbf{\Gamma} \mathbf{\Lambda} \mathbf{\Gamma}^T
    $$
    
        New coordinate system
    $$\mathbf{\Gamma}$$

        Variance of each direction
        Covariance matrix on the new coordinate
    $$\mathbf{\Lambda}$$
    $$

    Transformed dataset
$$\mathbf{Y}=\mathbf{X}\mathbf{\Gamma}$$
    Covariance matrix of transformed dataset
$$\mathbf{\Sigma_\mathbf{Y}}=\mathbf{\Lambda}=\mathbf{\Gamma}^T{\mathbf{\Sigma_{\hat{\mathbf{X}}}}}\mathbf{\Gamma}$$

$$

$$